(unit2:randomquiz)=
# Unit 2 : Random Quiz
$\,\!$

The quiz below is a formative assessment of the topics in unit 2. Questions are randomly shuffled and only twenty are shown at any one time so refreshing the page can generate a new set of questions. These are **not exam style questions**, but should cover learning the basic understanding of terms used in the course.

In [ ]:
import micropip
await micropip.install('jupyterquiz')
from jupyterquiz import display_quiz
display_quiz('phy303-unit2-quiz-answers.json', shuffle_questions=True, num=20)

# Unit 2 : Calculations Quiz

The calculation quiz below instead allows numeric answer inputs to a number of standard calculations covered in unit 2. These are generated dynamically for a range of inputs, and therefore only a subset are included in the full length quiz. Refresh the page to generate a new set of questions.

The following questions use the SEMF and assume the same constants for the SEMF as those in the Unit 2 Equation and terms section.

In [ ]:
import json
from jupyterquiz import display_quiz

import numpy as np
import pandas as pd

livechart = "https://nds.iaea.org/relnsd/v0/data?"
import urllib.request
def lc_pd_dataframe(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))
df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all")
df["a"]=df["z"] + df["n"]
df["b_over_a"] = df["binding"]


df["Atomic Number [A]"] = df["z"]  + df["a"]
df["Proton Number [Z]"] = df["z"]
df["Neutron Number [N]"] = df["n"]
df["Binding Energy [MeV]"] = df["binding"] / 1000.0
df["Symbol"] = df["symbol"] 
df["Dominant Decay"] = df["decay_1"] 


def volume(A):
    return 15.56 * A

def surface(A):
    return -17.23 * A**(2.0/3.0)

def coulomb(A,Z):
    return -0.72*Z*(Z-1) / A**(1.0/3.0)

def assym(A,Z):
    p_aa = 93.14
    B_a=-p_aa*(Z-A/2)**2/A
    return B_a

def pair(A,Z):
    p_ap=12.0
    B_p=-p_ap/A**(1/2) 
    N=A-Z
    
    CV = np.array(Z%2 + N%2)
    B_p = np.array(B_p)
    B_p[ CV == 1 ] = -B_p[ CV == 1 ]
    B_p[ CV == 0 ] = 0

    return B_p

df["semf"] = (volume(df.a)+assym(df.a, df.z) + pair(df.a, df.z) + coulomb(df.a, Z=np.array(df.z)) + surface(df.a))/df.a/1E-3
df["LDM Ratio"] = df["semf"] / df["binding"]

df["radius"] = 1.21 * np.pow(df.a,1./3.)
df["protonsep1"] = 0.0
df["protonsep2"] = 0.0
df["neutronsep1"] = 0.0
df["neutronsep2"] = 0.0

allqs = []


for i in range(200):

    iso = df.sample(1)
    nuclei = iso["symbol"].values[0]
    protons = iso["z"].values[0]
    neutrons = iso["n"].values[0]

    # SEMF Calculation
    qss = {
        "question": f"Determine the total binding energy in MeV for {nuclei} which has {protons} protons and {neutrons} neutrons.",
        "type": "numeric",
        "precision": 4,
        "answers": []
    }

    value = iso.semf.values[0]

    qss["answers"].append(
        {
            "type": "value",
            "value": value,
            "correct": True,
            "feedback": "Correct!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [value*0.9, value*1.1],
            "correct": True,
            "feedback": "Correct within 10%!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [-1000000000000, 0],
            "correct": False,
            "feedback": "Binding energies are always expressed as positive in MeV"
        }
    )

    qss["answers"].append(
        {
            "type": "default",
            "feedback": "Incorrect, check your constants and Np and Nn!"
        }
    )

    allqs.append(qss)


    # Nuclear Radius
    qss = {
        "question": f"Assuming a solid sphere, estimate the nuclear radius in fm for {nuclei} which has {protons} protons and {neutrons} neutrons.",
        "type": "numeric",
        "precision": 4,
        "answers": []
    }

    value = iso.radius.values[0]

    qss["answers"].append(
        {
            "type": "value",
            "value": value,
            "correct": True,
            "feedback": "Correct!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [value*0.9, value*1.1],
            "correct": True,
            "feedback": "Correct within 10%!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [-1000000000000, 0],
            "correct": False,
            "feedback": "Nuclear radii are always positive."
        }
    )

    qss["answers"].append(
        {
            "type": "default",
            "feedback": "Incorrect, check your constants and Np and Nn!"
        }
    )

    allqs.append(qss)

    # SEMF 1n Calculation
    qss = {
        "question": f"Determine the single neutron seperation energy in MeV for {nuclei} which has {protons} protons and {neutrons} neutrons.",
        "type": "numeric",
        "precision": 4,
        "answers": []
    }

    value = iso.neutronsep1.values[0]

    qss["answers"].append(
        {
            "type": "value",
            "value": value,
            "correct": True,
            "feedback": "Correct!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [value*0.9, value*1.1],
            "correct": True,
            "feedback": "Correct within 10%!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [-1000000000000, 0],
            "correct": False,
            "feedback": "Binding energies are always expressed as positive in MeV"
        }
    )

    qss["answers"].append(
        {
            "type": "default",
            "feedback": "Incorrect, check your constants and Np and Nn! Are you getting the single neutron seperation?"
        }
    )

    allqs.append(qss)

    # SEMF 2n Calculation
    qss = {
        "question": f"Determine the double neutron seperation energy in MeV for {nuclei} which has {protons} protons and {neutrons} neutrons.",
        "type": "numeric",
        "precision": 4,
        "answers": []
    }

    value = iso.neutronsep2.values[0]

    qss["answers"].append(
        {
            "type": "value",
            "value": value,
            "correct": True,
            "feedback": "Correct!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [value*0.9, value*1.1],
            "correct": True,
            "feedback": "Correct within 10%!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [-1000000000000, 0],
            "correct": False,
            "feedback": "Binding energies are always expressed as positive in MeV"
        }
    )

    qss["answers"].append(
        {
            "type": "default",
            "feedback": "Incorrect, check your constants and Np and Nn! Are you getting the double neutron seperation?"
        }
    )

    allqs.append(qss)

    # SEMF 1p Calculation
    qss = {
        "question": f"Determine the single proton seperation energy in MeV for {nuclei} which has {protons} protons and {neutrons} neutrons.",
        "type": "numeric",
        "precision": 4,
        "answers": []
    }

    value = iso.protonsep1.values[0]

    qss["answers"].append(
        {
            "type": "value",
            "value": value,
            "correct": True,
            "feedback": "Correct!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [value*0.9, value*1.1],
            "correct": True,
            "feedback": "Correct within 10%!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [-1000000000000, 0],
            "correct": False,
            "feedback": "Binding energies are always expressed as positive in MeV"
        }
    )

    qss["answers"].append(
        {
            "type": "default",
            "feedback": "Incorrect, check your constants and Np and Nn! Are you getting the single proton seperation?"
        }
    )

    allqs.append(qss)

    # SEMF 2p Calculation
    qss = {
        "question": f"Determine the single proton seperation energy in MeV for {nuclei} which has {protons} protons and {neutrons} neutrons.",
        "type": "numeric",
        "precision": 4,
        "answers": []
    }

    value = iso.protonsep1.values[0]

    qss["answers"].append(
        {
            "type": "value",
            "value": value,
            "correct": True,
            "feedback": "Correct!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [value*0.9, value*1.1],
            "correct": True,
            "feedback": "Correct within 10%!"
        }
    )

    qss["answers"].append(
        {
            "type": "range",
            "range": [-1000000000000, 0],
            "correct": False,
            "feedback": "Binding energies are always expressed as positive in MeV"
        }
    )

    qss["answers"].append(
        {
            "type": "default",
            "feedback": "Incorrect, check your constants and Np and Nn! Are you getting the double proton seperation?"
        }
    )

    allqs.append(qss)


with open("phy303-unit2-dynamicquestion-answers.json", "w") as f:
    json.dump(allqs, f, indent=4)

#
display_quiz('phy303-unit2-dynamicquestion-answers.json', shuffle_questions=True, num=10)